# Training of the Phonetic model

In [1]:
from src.pairing.dataset.phonetic_pair_dataset import PhoneticPairDataset
from src.pairing.dataset.phonetic_triplet_dataset import PhoneticTripletDataset
from src.pairing.model.phonetic_siamese import PhoneticSiamese
from src.pairing.training.config import CONFIG, LossType
from torch.utils.data import Dataset
from pathlib import Path

from pytorch_lightning.loggers import MLFlowLogger
from pytorch_lightning.utilities.seed import seed_everything
from torch.utils.data import DataLoader
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from jina import DocumentArray, Document
from pathlib import Path

import pandas as pd
import mlflow
import torch

seed_everything(0)

c:\Users\simon\anaconda3\envs\geometric\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 0


0

In [2]:
def get_dataset() -> Dataset:
    """Returns a Dataset object given loss type."""
    if CONFIG.loss_type == LossType.Pair:
        dataset = PhoneticPairDataset(
            best_pairs_path=CONFIG.best_pairs_dataset, worst_pairs_path=CONFIG.worst_pairs_dataset
        )
    elif CONFIG.loss_type == LossType.Triplet:
        dataset = PhoneticTripletDataset(
            best_pairs_path=CONFIG.best_pairs_dataset, worst_pairs_path=CONFIG.worst_pairs_dataset
        )
    elif CONFIG.loss_type == LossType.Mixed:
        dataset = PhoneticTripletDataset(
            best_pairs_path=CONFIG.best_pairs_dataset, worst_pairs_path=CONFIG.worst_pairs_dataset
        )
    else:
        raise ValueError(f'Unknown loss type given: {CONFIG.loss_type}')
    return dataset

dataset = get_dataset()

In [3]:
# third = len(dataset)//3
# train_set, val_set, test_set = torch.utils.data.random_split(
#     dataset, [third, third, len(dataset)-2*third]
# )
train_set, val_set, test_set = torch.utils.data.random_split(
    dataset, [len(dataset)-600, 300, 300]
)

In [4]:
def instanciate(kwargs):
    train_dataloader = DataLoader(
        train_set, batch_size=kwargs["batch_size"], shuffle=True, num_workers=4
    )
    validation_dataloader = DataLoader(
        val_set, batch_size=kwargs["batch_size"], num_workers=4
    )
    test_dataloader = DataLoader(
        test_set, batch_size=kwargs["batch_size"], num_workers=4
    )
    model = PhoneticSiamese(
        embedding_dim=kwargs["embedding_dim"],
        dim_feedforward=kwargs["dim_feedforward"],
        nhead=kwargs["nhead"],
        dropout=kwargs["dropout"],
        loss_type=CONFIG.loss_type,
        batch_size=kwargs["batch_size"],
        weight_decay=kwargs["weight_decay"],
        lr=kwargs["lr"],
        margin=kwargs["margin"],
        lambda_triplet=kwargs["lambda_triplet"],
        lambda_pos=kwargs["lambda_pos"],
        lambda_neg=kwargs["lambda_neg"],
    )
    return {
        "train_dataloader": train_dataloader,
        "validation_dataloader": validation_dataloader,
        "test_dataloader": test_dataloader,
        "model": model,
    }

In [5]:
def train(
        dropout,
        lr,
        weight_decay,
        dim_feedforward,
        batch_size,
        nhead,
        embedding_dim,
        margin
):
    mlf_logger = MLFlowLogger(
        experiment_name=CONFIG.experiment_name, tracking_uri=CONFIG.log_folder
    )
    trainer = Trainer(
        max_epochs=CONFIG.max_epochs,
        logger=mlf_logger,
        # callbacks=[EarlyStopping(monitor="validation_loss", mode="min")],
        accelerator="gpu", 
        devices=1
    )
    instance = instanciate(
        {
            "dropout": dropout,
            "lr": lr,
            "weight_decay": weight_decay,
            "dim_feedforward": dim_feedforward,
            "batch_size": batch_size,
            "nhead": nhead,
            "embedding_dim": embedding_dim,
            "margin": margin,
            "model": "phonetic_siamese",
            "lambda_triplet": 0.5,
            "lambda_pos": 0.5,
            "lambda_neg": 0,
        }
    )

    mlflow.pytorch.autolog()

    with mlflow.start_run():
        model = fit_model(
            instance["model"],
            instance["train_dataloader"],
            instance["validation_dataloader"],
            trainer,
        )

        test_loss = test_model(model, instance["test_dataloader"], trainer)[0]["test_loss"]

        torch.save(model.state_dict(), "model_dict")
        torch.save(model.state_dict(), "../src/pairing/model/model_dict")
        mlflow.log_artifact("model_dict", "model_dict")

    return model, test_loss

def fit_model(model, train_dataloader, validation_dataloader, trainer):
    trainer.fit(model, train_dataloader, validation_dataloader)
    return model

def test_model(model, test_dataloader, trainer):
    return trainer.test(model, test_dataloader, verbose=False)

In [6]:
model, test_loss = train(
    dropout=0.2,
    lr=1e-3,
    weight_decay=1e-3,
    dim_feedforward=16,
    batch_size=16,
    nhead=2,
    embedding_dim=16,
    margin=0.2
)

print(f'Final test loss: {test_loss}')

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type                    | Params
---------------------------------------------------------------
0 | embedding          | Embedding               | 9.4 K 
1 | encoder            | TransformerEncoderLayer | 1.7 K 
2 | cos                | CosineSimilarity        | 0     
3 | p_enc_1d_model     | PositionalEncoding1D    | 0     
4 | p_enc_1d_model_sum | Summer                  | 0     
5 | triplet_loss       | TripletMarginLoss       | 0     
---------------------------------------------------------------
11.1 K    Trainable params
0         Non-trainable params
11.1 K    Total params
0.044     Total estimated model params size (MB)


Epoch 24: 100%|██████████| 2759/2759 [00:57<00:00, 48.11it/s, loss=0.0218, v_num=3785, validation_loss=0.0213, training_loss=0.0166] 


2023/03/03 22:17:43 WARNING mlflow.utils.requirements_utils: Found torch version (1.13.1+cu116) contains a local version label (+cu116). MLflow logged a pip requirement for this package as 'torch==1.13.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2023/03/03 22:17:50 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\simon\anaconda3\envs\geometric\lib\site-packages\pkg_resources\_vendor\packaging\specifiers.py:255: DeprecationWarning: Creating a LegacyVersion has been deprecated and will be removed in the next major release"
2023/03/03 22:17:50 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\simon\anaconda3\envs\geometric\lib\site-packages\pkg_resources\_vendor\packaging\specifiers.py:255: DeprecationWarning: Creating a LegacyVersion has been deprecated and will be removed i

Testing DataLoader 0: 100%|██████████| 19/19 [00:00<00:00, 160.33it/s]
Final test loss: 0.021190408617258072


## Ranking Evaluation

In [7]:
model = model.eval()

In [8]:
model = model.to("cuda:0")

def load_documents() -> DocumentArray:
    dataframe = pd.read_csv(Path("../src/pairing/dataset/pairing/english.csv"))
    words = dataframe[['word', 'ipa']].astype(str)  

    local_da = DocumentArray([Document(text=w['word'], ipa=w['ipa']) for _, w in words.iterrows()])
    def embed(da: DocumentArray) -> DocumentArray:
            x = da[:,'tags__ipa']
            da.embeddings = model.encode(x).detach().cpu() 
            return da
    local_da.apply_batch(embed, batch_size=32)

    with DocumentArray() as da:
        da += local_da
    return da

In [9]:
def evaluate(da, n_limit=10):
    sample_data = dataset.best_pairs.sample(100)

    queries = sample_data['ipa_a']
    query_text = sample_data['word_a']
    targets = sample_data['word_b']

    # 1. Generate all embeddings for pairs.
    query_embeddings = model.encode(queries).detach().cpu() 
    query_docs = DocumentArray([Document(text=text, ipa=ipa, embedding=embedding) for ipa, text, embedding in zip(queries, query_text, query_embeddings)])

    # 2. For each pair: query the da
    [
        doc.match(da, metric='euclidean', limit=n_limit) for doc in query_docs
    ]

    matches = [
        doc.matches[:,'text'] for doc in query_docs
    ]

    # 3. For each pair: check if targetted word is amongst results
    scores = [
        t in match for t, match in zip(targets, matches)
    ]
    return scores


In [10]:
da = load_documents()

scores = evaluate(da, n_limit=5)
print(f"Percentage of the original pair that have been retrieved: {sum(scores) / len(scores)*100} %")

Percentage of the original pair that have been retrieved: 6.0 %


## Inference example

In [ ]:
device = torch.device('cuda:0')

In [ ]:
import numpy as np
model.eval().to(device)

for i in np.random.randint(0, len(test_set), 10):
    sample = test_set[i]
    anchor_match = sample['anchor_phonetic']
    positive_match = sample['similar_phonetic']
    negative_match = sample['distant_phonetic']

    anchor_embedding = model.encode([anchor_match])
    positive_embedding = model.encode([positive_match])
    negative_embedding = model.encode([negative_match])

    loss = model.triplet_loss(anchor_embedding, positive_embedding, negative_embedding)

    positive_dist = torch.cdist(anchor_embedding, positive_embedding, p=2)
    negative_dist = torch.cdist(anchor_embedding, negative_embedding, p=2)

    print(f"Loss: {loss}")
    print(f"Positive: {positive_dist}")
    print(f"Negative: {negative_dist}")

In [ ]:
from eng_to_ipa import convert
words = ['dog', 'parade', 'cascade', "palace", "table"]
ipas = [convert(x) for x in words]

device = torch.device('cuda:0')
model.eval().to(device)
embdedings = model.encode(ipas)

a = torch.cdist(embdedings[1].view(1, -1), embdedings[0].view(1, -1), p=2)
b = torch.cdist(embdedings[1].view(1, -1), embdedings[2].view(1, -1), p=2)
c = torch.cdist(embdedings[1].view(1, -1), embdedings[3].view(1, -1), p=2)
d = torch.cdist(embdedings[1].view(1, -1), embdedings[4].view(1, -1), p=2)

a, b, c, d